In [1]:
import numpy as np

In [1]:
class KNearestNeighbour():
    def __init__(self,K):
        self.K = K

    def train(self,X,y):
        self.X_train = X
        self.y_train = y

    def predict(self,X_test):
        distances = self.compute_distance(X_test) 
        return self.predict_labels(distances)

    def compute_distance(self,X_test):
        num_test = X_test.shape[0]
        num_train = self.X_train.shape[0]
        distances = np.zeros((num_test,num_train))

        for i in range(num_test):
            for j in range(num_train):
                distances[i,j] = np.sqrt(np.sum((X_test[i,:] - self.X_train[j,:])**2))

        return distances

    def predict_labels(self,distances):
        num_test = distances.shape[0]
        y_pred = np.zeros(num_test)

        for i in range(num_test):
            # returns indices according to value
            # eg; [18, 68,  8, 78, 88] -> array([2, 0, 1, 3, 4], dtype=int64)
            y_indices = np.argsort(distances[i,:]) 
            k_closest_classes = self.y_train[y_indices[:self.K]].astype(int)

            # [1,1,3,4,3,3] -> [0,2,0,3,1]
            y_pred[i] = np.bincount(k_closest_classes)

In [ ]:
if __name__ == "main":
    pass